In [5]:
!pip install langchain
!pip install langgraph
!pip install -qU langchain[google-genai]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 151.9/151.9 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.6/50.6 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.5/216.5 kB 2.1 MB/s eta 0:00:00


In [6]:
import getpass
import os

if not os.environ.get("GOOGLE_API_KEY"):
  os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter API key for Google Gemini: ")

from langchain.chat_models import init_chat_model

model = init_chat_model("gemini-2.0-flash", model_provider="google_genai")

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
chat_prompt_template = ChatPromptTemplate.from_messages(
    [("system", "You are a 10/10 chinese teacher, teaching native speakers of english. You teach people a little at a time and talk as if you are having a conversation and not a lecture",), MessagesPlaceholder(variable_name="messages"),]
)

In [ ]:
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph

# Define a new graph
workflow = StateGraph(state_schema=MessagesState)

def call_model(state: MessagesState):
    prompt = chat_prompt_template.invoke(state["messages"])
    response = model.invoke(prompt)
    return {"messages": response}

# Define the (single) node in the graph
workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

#Adding Memory
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [10]:
from langchain_core.messages import HumanMessage

config = {"configurable": {"thread_id": "CM"}}
while True:
  user_input = input("You>:")
  input_messages = [HumanMessage(user_input)]
  output = app.invoke({"messages": input_messages}, config)
  print("Teacher>:", end="")
  output["messages"][-1].pretty_print()

You>:Hi can you teach me some chinese
Teacher>:================================== Ai Message ==================================

你好! (Nǐ hǎo!) Hello! I'm ready to teach you some Chinese. To make this effective, tell me:

1.  **What are your goals?**  Do you want to be able to order food, greet people, have basic conversations, or something else?
2.  **What's your current level?**  Have you studied any Chinese before? Do you know any Pinyin?
3.  **What's your learning style?** Do you prefer learning phrases, grammar rules, or a mix of both?

In the meantime, let's start with a super basic phrase:

**谢谢 (xiè xiè) - Thank You**

*   **Pinyin:** xiè xiè
*   **Pronunciation:**
    *   **xiè:** This is the 4th tone. This is a sharp, falling tone. Imagine you're saying "Shh!" to someone. It should be quick and decisive. xiè, xiè, xiè.
    *   **xiè:** Same as above.
*   **Meaning:** Thank you.
*   **Cultural Note:** It's common to repeat the word "xiè" for emphasis. So "xiè xiè" is the standa

KeyboardInterrupt: Interrupted by user